In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import requests

API_KEY = "644499f581c7474c88322946243108"
CITY = "salem"
API_URL = f"http://api.weatherapi.com/v1/forecast.json?key={API_KEY}&q={CITY}&days=2&aqi=no"

def fetch_weather_data(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: Failed to fetch data from the API. Status code: {response.status_code}")
        return None

def parse_weather_data(data):
    try:
        # Current weather
        current_temp = data['current']['temp_c']
        current_humidity = data['current']['humidity']
        current_rainfall = data['current'].get('precip_mm', 0)
        current_wind_speed = data['current'].get('wind_kph', 0)

        # Forecast for the next day
        next_day_temp = data['forecast']['forecastday'][1]['day']['avgtemp_c']
        next_day_humidity = data['forecast']['forecastday'][1]['day']['avghumidity']
        next_day_rainfall = data['forecast']['forecastday'][1]['day']['totalprecip_mm']
        next_day_wind_speed = data['forecast']['forecastday'][1]['day']['maxwind_kph']

        return (current_temp, current_humidity, current_rainfall, current_wind_speed,
                next_day_temp, next_day_humidity, next_day_rainfall, next_day_wind_speed)
    except KeyError as e:
        print(f"Error: Missing key {e} in the API response.")
        return None

def train_and_predict(features, data):
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(data[features], data['next_day_temperature'])
    data['predicted_next_day_temperature'] = model.predict(data[features])
    mae = mean_absolute_error(data['next_day_temperature'], data['predicted_next_day_temperature'])
    mse = mean_squared_error(data['next_day_temperature'], data['predicted_next_day_temperature'])
    print(f"Next Day Temperature Prediction MAE: {mae:.2f}, MSE: {mse:.2f}")
    return model

def main():
    # Fetch data
    data = fetch_weather_data(API_URL)
    
    if data:
        # Parse data
        (current_temp, current_humidity, current_rainfall, current_wind_speed,
         next_day_temp, next_day_humidity, next_day_rainfall, next_day_wind_speed) = parse_weather_data(data)

        if next_day_temp is not None:
            # Create DataFrame
            data_dict = {
                'current_temperature': [current_temp],
                'current_humidity': [current_humidity],
                'current_rainfall': [current_rainfall],
                'current_wind_speed': [current_wind_speed],
                'next_day_temperature': [next_day_temp],
                'next_day_humidity': [next_day_humidity],
                'next_day_rainfall': [next_day_rainfall],
                'next_day_wind_speed': [next_day_wind_speed]
            }
            data = pd.DataFrame(data_dict)

            print(data.head())

            # Define features
            features = ['current_temperature', 'current_humidity', 'current_rainfall', 'current_wind_speed']

            # Train model and predict
            model = train_and_predict(features, data)

            # Display results
            print(data[['next_day_temperature', 'predicted_next_day_temperature']])
        else:
            print("Error: Could not retrieve weather data properly.")
    else:
        print("Error: Could not fetch data from the API.")

main()

   current_temperature  current_humidity  current_rainfall  \
0                 24.9                86              0.71   

   current_wind_speed  next_day_temperature  next_day_humidity  \
0                 3.6                  24.6                 84   

   next_day_rainfall  next_day_wind_speed  
0              14.12                  6.8  
Next Day Temperature Prediction MAE: 0.00, MSE: 0.00
   next_day_temperature  predicted_next_day_temperature
0                  24.6                            24.6
